# Simple implementation of the BM25 algorithm in python

* https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables/
* https://cs.uwaterloo.ca/~jimmylin/publications/Kamphuis_etal_ECIR2020_preprint.pdf

In [15]:
# !pip install bm25s

In [56]:
import pandas as pd

df = pd.read_csv("data/corpus.csv")
documents = df["text"].tolist()

In [58]:
import bm25s
corpus_tokens = bm25s.tokenize(documents, stopwords="en")
retriever = bm25s.BM25(method="lucene")
retriever.index(corpus_tokens)

Split strings:   0%|          | 0/30000 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/30000 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/30000 [00:00<?, ?it/s]

In [60]:
query = "amore classico 1.75"
query_tokens = bm25s.tokenize(query)
print(query_tokens)
results, scores = retriever.retrieve(query_tokens, corpus=documents, k=10)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Tokenized(ids=[[0, 1, 2]], vocab={'amore': 0, 'classico': 1, '75': 2})


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 5.69): Amaretto Di Amore Classico
Rank 2 (score: 3.53): Bell'Amore Peach
Rank 3 (score: 3.20): Di Amore Raspberry 33
Rank 4 (score: 3.20): Di Amore Amaretto 42
Rank 5 (score: 3.20): Primo Amore Moscato Puglia
Rank 6 (score: 3.20): Di Amore Amaretto 42
Rank 7 (score: 3.20): Di Amore Amaretto 42
Rank 8 (score: 3.20): Bell'Amore Peach (Sc)
Rank 9 (score: 3.20): Di Amore Sambuca 84
Rank 10 (score: 3.20): Di Amore Amaretto 42


In [63]:
import re

TOKEN_PATTERN = r"(?u)\b\w\w+\b"
STOPWORDS = (
    "a",
    "an",
    "and",
    "are",
    "as",
    "at",
    "be",
    "but",
    "by",
    "for",
    "if",
    "in",
    "into",
    "is",
    "it",
    "no",
    "not",
    "of",
    "on",
    "or",
    "such",
    "that",
    "the",
    "their",
    "then",
    "there",
    "these",
    "they",
    "this",
    "to",
    "was",
    "will",
    "with",
)

def tokenize(text: str) -> list[str]:

    split_fn = re.compile(TOKEN_PATTERN).findall

    words = split_fn(text)
    words = [word.lower().strip() for word in words]
    words = [word for word in words if word not in STOPWORDS]
    return words

tokenize("Orange Amaretto")

['orange', 'amaretto']

In [65]:
from collections import defaultdict

# f(qi) - document_freq is the number of documents which contain the token term
document_freq = defaultdict(int)
# f(dj, qi) - term_freq is the number of times the token term appears in document dj
term_freq = defaultdict(int)

vocab = []
doc_lengths = []

for doc_id, text in enumerate(documents):

    tokens = tokenize(text)
    doc_lengths.append(len(tokens))
    vocab.extend(tokens)

    for token in set(tokens):
        document_freq[token] += 1

    for token in tokens:
        term_freq[(doc_id, token)] += 1

vocab = sorted(set(vocab))

doc_count = len(documents)
mean_doc_length = sum(doc_lengths) / doc_count

doc_count, mean_doc_length, len(vocab), len(document_freq), len(term_freq)

(30000, 5.5728, 10204, 10204, 165624)

In [72]:
import math


def score_idf(token: str) -> float:
    fq = document_freq[token]
    inner = (doc_count - fq + 0.5) / (fq + 0.5)
    if inner < 1:
        inner = 1
    return math.log(inner)


def score_tf(doc_id: int, token: str, k1 = 1.5, b = 0.75) -> float:
    doc_length = doc_lengths[doc_id]
    f = term_freq[(doc_id, token)]
    return f / (f + k1 * (1 - b + b * doc_length / mean_doc_length))


token = vocab[0]
score_tf(0, token), score_idf(token)

(0.0, 8.168636465892812)

In [76]:
import numpy as np


def score_query(token: str) -> np.ndarray:
    document_scores = []
    for doc_id in range(doc_count):
        score = score_idf(token) * score_tf(doc_id, token)
        document_scores.append(score)
    return np.array(document_scores)

In [82]:
query = "amore classico"

scores = np.zeros([doc_count])
for token in tokenize(query):
    scores += np.array(score_query(token))

top_k = 10
top_k_docs = np.argsort(scores)[::-1][:top_k]
top_k_scores = scores[top_k_docs]

for i, (doc_id, score) in enumerate(zip(top_k_docs, top_k_scores)):
    print(f"Rank {i+1} (score: {score:.2f}): {documents[doc_id]}")

Rank 1 (score: 5.68): Amaretto Di Amore Classico
Rank 2 (score: 3.53): Bell'Amore Peach
Rank 3 (score: 3.20): Di Amore Limoncello 53
Rank 4 (score: 3.20): Primo Amore Moscato Puglia
Rank 5 (score: 3.20): Di Amore Sambuca 84
Rank 6 (score: 3.20): Di Amore Sambuca 84
Rank 7 (score: 3.20): Di Amore Sambuca 84
Rank 8 (score: 3.20): Di Amore Sambuca 84
Rank 9 (score: 3.20): Di Amore Sambuca 84
Rank 10 (score: 3.20): Di Amore Raspberry 33


In [83]:
query_tokens = bm25s.tokenize(query)
results, scores = retriever.retrieve(query_tokens, corpus=documents, k=10)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {doc}")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Rank 1 (score: 5.69): Amaretto Di Amore Classico
Rank 2 (score: 3.53): Bell'Amore Peach
Rank 3 (score: 3.20): Terra Valentine Amore 14
Rank 4 (score: 3.20): Di Amore Sambuca 84
Rank 5 (score: 3.20): Di Amore Amaretto 42
Rank 6 (score: 3.20): Di Amore Sambuca 84
Rank 7 (score: 3.20): Di Amore Amaretto 42
Rank 8 (score: 3.20): Di Amore Sambuca 84
Rank 9 (score: 3.20): Di Amore Sambuca 84
Rank 10 (score: 3.20): Di Amore Limoncello 53
